# LLM에서의 Few-shot Prompting
**Few-shot prompting**은 GPT-3, GPT-4와 같은 대규모 언어 모델(LLM)에 소수(보통 1~5개)의 예시를 제공해, 모델의 반응을 안내하거나 특정 작업을 가르치는 기술입니다.  
이는 **프롬프트 엔지니어링(Prompt Engineering)**이라는 더 넓은 범위의 기술 중 하나로, AI 모델과 효과적으로 소통하기 위한 입력(프롬프트)을 설계하는 “예술”이라고 할 수 있습니다.

Few-shot prompting에서 모델에게 주어진 예시들은 패턴을 보여주거나 특정 반응 방식을 가르치기 위해 신중하게 선택됩니다. 모델은 이 예시들을 참조해 자신의 반응에서 기대되는 것을 이해하게 됩니다.  

이 방식은 단순 지시만으로는 의도를 충분히 전달하기 어려운 **복잡하거나 미묘한 작업**에 특히 유용합니다.

## Few-shot Prompting 사용 방법 
1. 몇 개의 예시를 제공하여 LLM 모델에게 특정 출력을 지시하는 Few-shot Prompting
2. 사용자 인풋과의 유사도에 따라 예시를 선택해서 LLM 모델에 Few-show Prompting
---

## ✅ Few-shot Prompting의 주요 포인트
- **예시를 통한 시연 (Demonstration through Examples)**  
  특정 예시를 제공함으로써 모델에게 어떤 종류의 출력이 기대되는지 보여줍니다. 분류, 번역, 창의적 글쓰기 등 다양한 작업에 효과적입니다.

- **모호성 감소 (Reduction in Ambiguity)**  
  작업 지시에서 모호성을 줄여, 모델에게 무엇을 해야 하는지 더 명확하게 전달합니다.

- **유연성과 적응성 (Flexibility and Adaptability)**  
  별도의 재학습이나 모델 업데이트 없이도, 새로운 작업에 쉽게 적응할 수 있습니다.

- **효율성 (Efficiency)**  
  대량의 데이터나 광범위한 파인튜닝 없이도 새로운 작업에 모델을 적용할 수 있어 매우 효율적입니다.

- **예시 품질에 대한 의존성 (Dependency on Example Quality)**  
  Few-shot prompting의 효과는 예시의 품질과 관련성에 크게 좌우됩니다.  
  잘못된 예시는 모델의 반응을 부적절하게 만들 수 있으니, 예시 선택이 중요합니다.

---

## 💡 요약

Few-shot prompting은 **Finetuning**이나 대규모 데이터셋 준비가 힘든 환경에서,  
**LLM의 능력을 유연하고 효율적으로 끌어내는 강력한 도구**입니다.


In [2]:
# 더 많은 예제 추가
examples = [
    {"input": "기쁨", "output": "슬픔"},
    {"input": "높음", "output": "낮음"},
    {"input": "빠름", "output": "느림"},
    {"input": "밝음", "output": "어둠"},
    {"input": "강함", "output": "약함"},
    {"input": "크다", "output": "작다"},
    {"input": "덥다", "output": "춥다"},
    {"input": "좋음", "output": "나쁨"},
    {"input": "오름", "output": "내림"},
    {"input": "많음", "output": "적음"},
]

In [3]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

# 벡터 저장소 생성
to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)
# 예제 선택기 생성
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=4,
)

In [4]:
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

# Few-shot 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)
print(few_shot_prompt.format(input="건조하다"))

Human: 크다
AI: 작다
Human: 덥다
AI: 춥다
Human: 강함
AI: 약함
Human: 많음
AI: 적음


In [5]:
# 최종 프롬프트 템플릿 생성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 한국어 백과사전입니다. 어떤 단어가 들어와도 그에 대한 반의어를 알려줍니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [9]:
from langchain_openai import ChatOpenAI

# 모델과 체인 생성
chain = final_prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 질문하기
result = chain.invoke("뚱뚱")
print(result.content)

날씬


## 다른 예제 - 추론 유도하기
FewShotPromptTemplate를 이용해 입력에 대한 결과 형식 지정해주기 

In [5]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
# 프롬프트 입력과 결과, 이를 통해 추론과정을 확인할 수 있고 좀 더 논리적인 이유를 알 수 있다.
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# 결과 출력
chain = prompt | llm | StrOutputParser()
answer = chain.stream({"question" :"대구 시장의 나이와 이름은?" })
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: 현재 대구 시장은 누구인가요?  
중간 답변: 현재 대구 시장은 홍준표입니다.  
추가 질문: 홍준표는 몇 살인가요?  
중간 답변: 홍준표는 1950년 12월 5일에 태어났습니다. (2023년 기준으로 72세입니다.)  
최종 답변은: 홍준표, 72세

## VectorStore를 활용해 

In [6]:
examples = [
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": """
회의록: XYZ 회사 마케팅 전략 회의
일시: 2023년 12월 25일
장소: XYZ 회사 회의실
참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

1. 개회
   - 회의는 김수진 팀장의 개회사로 시작됨.
   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.

2. 시장 동향 개요 (김수진)
   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.
   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.

3. 디지털 마케팅 전략 (박지민)
   - 박지민은 디지털 마케팅 전략에 대해 발표.
   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.

4. 소셜 미디어 캠페인 (이준호)
   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.
   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.

5. 종합 논의
   - 팀원들 간의 아이디어 공유 및 토론.
   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.

6. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 박지민 담당.
""",
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
- 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.
- 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.
- 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.

이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다.
""",
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
    },
]
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

chroma = Chroma("fewshot_chat", OpenAIEmbeddings())

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 여기에는 선택 가능한 예시 목록이 있습니다.
    examples,
    # 여기에는 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스가 있습니다.
    OpenAIEmbeddings(),
    # 여기에는 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스가 있습니다.
    chroma,
    # 이것은 생성할 예시의 수입니다.
    k=1,
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
)
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        few_shot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)
# chain 생성
chain = final_prompt | llm

# 실행 및 결과 출력
answer = chain.stream(question)
stream_response(answer)

회의록: ABC 기술 회사 제품 개발 팀 주간 진행 상황 회의  
일시: 2023년 12월 26일  
장소: ABC 기술 회사 회의실  
참석자: 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)  

1. 개회  
   - 회의는 최현수 프로젝트 매니저의 개회사로 시작됨.  
   - 회의의 목적은 새로운 모바일 애플리케이션 프로젝트의 현재 진행 상황 검토 및 다가오는 마일스톤 계획 수립.

2. 진행 상황 업데이트  
   - **최현수 (프로젝트 매니저)**  
     - 전체 프로젝트 일정 및 마일스톤 검토.  
     - 현재 진행 상황이 계획에 부합하고 있음을 확인.  

   - **황지연 (주요 개발자)**  
     - 개발 진행 상황 보고.  
     - 주요 기능 구현 완료 및 버그 수정 작업 진행 중.  
     - 다음 주까지의 개발 목표 설정.  

   - **김태영 (UI/UX 디자이너)**  
     - UI/UX 디자인 진행 상황 업데이트.  
     - 사용자 피드백 반영하여 디자인 수정 작업 진행 중.  
     - 다음 주까지 최종 디자인 시안 제출 예정.

3. 목표 설정  
   - 팀은 다음 주까지의 목표를 설정.  
   - 각 팀원은 자신의 작업 영역에서의 구체적인 목표를 명확히 함.  
   - 다음 마일스톤에 대한 준비 사항 논의.

4. 기타 논의  
   - 프로젝트 관련 이슈 및 리소스 필요 사항 논의.  
   - 팀원 간의 협업 방안 및 커뮤니케이션 강화 방안 모색.

5. 마무리  
   - 다음 회의 일정 확정: 2024년 1월 2일.  
   - 회의록 정리 및 배포는 최현수 담당.  
   - 회의 종료.